# Complex case classification

## Introduction

In this notebook, we will perform sequential binary classification analysis using LLMs. Binary classification refers to the task of categorizing data entries into one of two predefined categories. Here, we explore a specific case where binary classifications are conducted sequentially, with the final classification outcome depending on the results of prior classifications.

To help you navigate this notebook, here is a step-by-step outline of what we will do:

1. **Getting started**  
   - Download and install the project and its dependencies, load import and your API key.

2. **Load and preprocess the dataset**  
   - Upload, explore and pre-process the dataset, with the sample dataset (recommended for a first use) or your own data.

3. **Prompt construction and classification on manually annotated data**  

4. **Evaluating Model Performance Against Human Annotations**  
   - Compute metrics (e.g., **Cohen's Kappa**, **Alt-Test**, ...)

5. **Final Step: Classify the Full Dataset**  

## Getting started

Before we begin, let's set up the environment by cloning the project and installing the necessary dependencies.

### Step 1: Clone the Project
Run the following cell to download the project files.
This will download the project folder into Colab and switch the working directory to it.

In [ ]:
!git clone https://github.com/OlivierLClerc/qualitative_analysis_project

### Step 2: Install Required Libraries
Now, install the project and its dependencies.

⚠️ Note:

- This will install all required libraries for the notebook to run.
- If Colab suggests restarting the runtime, click "Restart Runtime" and re-run this cell.

In [ ]:
%cd qualitative_analysis_project
%pip install .

### Step 3: Load Your OpenAI API Key

To use OpenAI models for analysis, you need to provide your **OpenAI API key**. This key allows secure access to the API.

#### Instructions

1. Click on the **🔑 "Key" icon** on the left sidebar in Colab (**⚙️ Settings** > **Secrets**).  
2. Click **"Add a new secret"**.  
3. Enter the following:  
   - **Name** → `OPENAI_API_KEY`  
   - **Value** → *Your OpenAI API Key* (Get it from [OpenAI](https://platform.openai.com/account/api-keys))  
4. Click **"Save"**.  

#### Troubleshooting

- **API Key not found?**  
  - Double-check that the secret name is exactly **`OPENAI_API_KEY`**.  
  - If the issue persists, **refresh the page** and rerun the cell.  

- **Is My Key Secure?**  
  - Yes! Colab's **Secrets Manager** encrypts your key and keeps it safe.  

In [ ]:
from google.colab import userdata
import os
import pandas as pd

# Retrieve API keys securely from Colab Secrets
API_KEY = userdata.get('OPENAI_API_KEY')

# Check if the API key was loaded
if API_KEY:
    print("✅ API Key loaded successfully!")
    os.environ['OPENAI_API_KEY'] = API_KEY
else:
    print("⚠️ API Key not found. Please check the Secrets panel.")

### Step 4: Import Project Modules

Now that the project is installed, let's import the necessary modules and functions from the `qualitative_analysis` package. These tools will help us load data, process text, and perform binary classification analysis.

In [ ]:
from qualitative_analysis import (
    load_data,
    clean_and_normalize,
    sanitize_dataframe,
)

from qualitative_analysis.scenario_runner import run_scenarios
from qualitative_analysis.evaluation import (
    compute_kappa_metrics,
    run_alt_test_on_results,
    compute_classification_metrics_from_results
)
from qualitative_analysis.metrics.krippendorff import (
    compute_krippendorff_non_inferiority,
    print_non_inferiority_results
)

c:\Users\ocler\miniconda3\envs\gpt_rl\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load and Preprocess the Dataset

In [2]:
# Define data directory
data_dir = 'data/complex_user_case'
os.makedirs(data_dir, exist_ok=True)

# Define the path to your dataset
data_file_path = os.path.join(data_dir, 'complex_data.xlsx')

# Load the data
data = load_data(data_file_path, file_type='xlsx', delimiter=';')

# Preview the data
data.head()

,name,Iteration,key,reference,IDENTIFY,GUESS,SEEK,ASSESS,identify_cues,guess_cues,...,Identify_validity,Guess_validity,Seek_validity,Assess_validity,mechanical_rating,Rater_Oli,Unvalid_Oli,Rater_Gaia,Unvalid_Gaia,Invalid_Gaia
0,aac13,3,aac13_3,"Toutankhamon était un pharaon, un roi de l'Egy...",L'Egypte antique,C'est un ancien pays de l'Afrique 💸,Qu'est ce que l'Égypte antique,Qui,"{""1"":""L'Egypte antique"",""2"":""Le principe de mo...","{""1"":""C'est un ancien pays de l'Afrique"",""2"":""...",...,1.0,1.0,1.0,NaN,NaN,NaN,False,NaN,False,False
1,aac13,4,aac13_4,La très grande majorité de l’or disponible dan...,Les composants électriques,Les composants électroniques sont par exemple ...,Qu'est-ce qu'une composants électroniques,Non,"{""1"":""Utilité de l'or pour les couronnes denta...","{""1"":""Les couronnes en or sont plus solides et...",...,3.0,NaN,NaN,NaN,NaN,NaN,False,NaN,False,False
2,aac13,5,aac13_5,Des scientifiques ont révélé l’existence de tr...,Des premiers humains l'Australie,C'est un pays du Sud,Où se trouve l'Australie,Oui,"{""1"":""La formation des traces dans les roches""...","{""1"":""Ces traces se forment automatiquement qu...",...,2.0,3.0,3.0,NaN,0.0,NaN,False,NaN,False,False
3,aac13,6,aac13_6,La religion de la Grèce antique comprend plusi...,Une mythologie,L'olympe est un endroit en Grèce,Qu'est ce qu'une mythologie,Non,"{""1"":""Une mythologie"",""2"":""Les autres mytholog...","{""1"":""Une mythologie est un ensemble de contes...",...,1.0,3.0,1.0,NaN,0.0,NaN,False,NaN,False,False
4,aac24,3,aac24_3,"Toutankhamon était un pharaon, un roi de l'Egy...",Toutânkhamon,Roi Pharaon,Quand les Pharaons sont-ils apparus,J'ai trouvé ma réponse,"{""1"":""L'Egypte antique"",""2"":""Le principe de mo...","{""1"":""C'est un ancien pays de l'Afrique"",""2"":""...",...,NaN,NaN,3.0,NaN,NaN,NaN,False,NaN,False,False


### Dataset Description

The dataset provided for this notebook is an anonymized subset from the study available at [X]. In the original experiment, children were tasked with reading a reference text and engaging in four sequential interactions with an interactive app. The goal of these steps was to help the children formulate a divergent question. A question is considered divergent if its answer is not explicitly stated in the reference text.

The four steps, include:
1. **Identify**: The child identifies a knowledge gap related to the reference text.
2. **Guess**: The child makes a guess about what the answer to the knowledge gap could be.
3. **Seek**: The child formulates a question to seek the answer.
4. **Assess**: The child evaluates whether the app provides an answer to their question.

This process is called a **cycle**. An annotator evaluates the validity of a cycle by answering a series of binary Yes/No questions (binary classifications). A cycle is deemed valid if all binary questions can be answered by "Yes"; otherwise, it is considered invalid. For more details, see the codebook provided in the prompt cell.

### Dataset Structure

The dataset includes the following key components:
- **ref**: A column containing the text that children were asked to read beforehand.
- **IDENTIFY**: A column containing the entries posed by the children during the Identify step.
- **GUESS**: A column containing the entries posed by the children during the Guess step.
- **SEEK**: A column containing the entries posed by the children during the Seek step.
- **ASSESS**: A column containing the entries posed by the children during the Assess step.

To classify a cycle, both the reference text and the entries from all four steps are required.

Additionally, the dataset includes ratings from two human annotators. These ratings enable us to compute inter-annotator agreement metrics, such as Cohen's kappa, to assess the reliability of the annotations once the analysis is complete.

### Data Preprocessing  (Optional, improve clarity and consistency of text data)

1. **Rename key columns**  
   Give important columns more descriptive names  
   (e.g. `ref` → `reference`).

2. **Clean textual data**  
   For each text column, run `clean_and_normalize(series)` to  
   - trim leading/trailing spaces  
   - convert accented characters to plain ASCII (e.g. `'é'` → `'e'`).

3. **Sanitize line breaks**  
   Run `sanitize_dataframe(df)` to replace newline (`\n`) and carriage‑return (`\r`) characters with a single space in every string column.

In [3]:
# 1a) Define a mapping from old column names to new names
rename_map = {
    "ref": "reference",
    "IDENTIFY": "identify",
    "GUESS": "guess",
    "SEEK": "seek",
    "ASSESS": "assess"
}

# 1b) Rename the columns in the DataFrame
data = data.rename(columns=rename_map)

# 2) Now define the new column names for cleaning
text_columns = ["reference", "identify", "guess", "seek", "assess"]

# 3) Clean and normalize the new columns
for col in text_columns:
    data[col] = clean_and_normalize(data[col])

# 4) Sanitize the DataFrame
data = sanitize_dataframe(data)


### Combine texts and questions

To prepare the data for the LLM, we gather exactly the information a human annotator would need—plus the **ID** so we can merge results back into the original DataFrame.  
The concatenated block of fields is called a **verbatim**.

#### Create the `verbatim` field

1. **Build verbatims**  
   For every row we create a multi‑line string containing:  
   - the respondent **ID**  
   - the cleaned **reference** text  
   - the five cleaned prompt‑response fields (**Identify**, **Guess**, **Seek**, **Assess**)  
   Each section is separated by a blank line for readability, and the result is written to a new column named `verbatim`.

2. **Sanity‑check**  
   - Print the total number of verbatims to ensure every row was processed.  
   - Display the first verbatim as a spot‑check of the format.

In [4]:
# Combine texts and entries

data['verbatim'] = data.apply(
    lambda row: (
        f"Id: {row['key']}\n\n"
        f"Text: {row['reference']}\n\n"
        f"Identify: {row['identify']}\n\n"
        f"Guess: {row['guess']}\n\n"
        f"Seek: {row['seek']}\n\n"
        f"Assess: {row['assess']}\n\n"
    ),
    axis=1
)

# Extract the list of verbatims
verbatims = data['verbatim'].tolist()

print(f"Total number of verbatims: {len(verbatims)}")
print(f"Verbatim example:\n{verbatims[0]}")

Total number of verbatims: 921
Verbatim example:
Id: aac13_3

Text: Toutankhamon etait un pharaon, un roi de l'Egypte antique. Il est tres connu aujourdhui parce que des archeologues ont retrouve son cercueil intacte avec tous ses tresors, en 1922. Pour les Egyptiens, il y avait une vie apres la mort, une vie eternelle. Cest pour cela que le corps devait etre conserve dans le meilleur etat possible, cest ce quon appelle la momification. Cest aussi pour cela que lon retrouve aujourdhui de la nourriture, des armes ou des tresors dans les tombeaux. Ces objets accompagnaient le pharaon dans sa vie apres la mort.

Identify: L'Egypte antique

Guess: C'est un ancien pays de l'Afrique 

Seek: Qu'est ce que l'Egypte antique

Assess: Qui




## Prompt construction and classification on manually annotated data

This framework allows you to evaluate different configurations to determine which prompt, model, and parameters yield the most accurate classification. These configurations are stored in the scenarios list.

The snippet defines two **classification scenarios** for evaluating participants’ “Identify → Guess → Seek → Assess” reasoning cycles with a Large Language Model (LLM).

Each scenario is a dictionary inside the `scenarios` list and can be seen as a self‑contained _experiment_: it specifies

* which LLM to call (`provider_llm1`, `model_name_llm1`, `temperature_llm1`);
* the **prompt template** that tells the LLM how to judge a single data row;
* the expected JSON output (fields listed in `selected_fields`);
* optional settings for **prompt‑refinement** by a second LLM (`provider_llm2`, …).

Running the pipeline iterates over every scenario and evaluates every (or a subsample of) data rows, then writes the chosen output fields back to your dataframe or file.

### LLM Settings

- `provider_llm1`: The LLM provider used for classification (`azure`, `openai`, `anthropic`, `gemini`)
- `model_name_llm1`: The model used for classification. This depends on the provider.

#### Example:

- **For** `azure` → `"gpt-4o"` or `"gpt-4o-mini"`
- **For** `openai` → `"gpt-4o"` or `"gpt-4o-mini"`
- **For** `anthropic` → `"claude-3-7-sonnet-20250219"`, `"claude-3-5-haiku-20241022"`
- **For** `gemini` → `"gemini-2.0-flash-001"`, `"gemini-2.5-pro-preview-03-25"`

- `temperature_llm1`: Controls output variability. Set to `0` for deterministic responses. Higher values add randomness (not recommended for evaluation tasks).
- `subsample_size`: Number of entries to evaluate. Set to `-1` to use the entire dataset.

### Prompt Configuration

- `prompt_name`: A short name identifying the scenario, used in performance tracking.
- `template`: The full prompt used to guide the LLM. It could include:
  - The **role** of the assistant
  - A **description** of the input columns
  - The **evaluation codebook** (la manière dont les données doivent etre classifiées)
  - Optionally, **examples**
  - ⚠️ **Must contain** the `{verbatim_text}` placeholder for the entry being evaluated

### Output

- `selected_fields`: The fields to extract from the LLM’s output (e.g., `"Classification"`, `"Reasoning"`).  
  You can modify this to include or exclude elements (like adding confidence scores, removing reasonning).
- `prefix`: The key to look for in the LLM output that contains the classification label (e.g., `"Classification"`).
Nous spécifions donc cela pour que le parsing du verdict soit plus facile, pour récuperer les labels de classification.
- `label_type`: Data type of the classification label. Typically `"int"` for binary classification (`0` or `1`),  
  but can be changed to `"float"` or `"str"` as needed.
- `response_template`: The required format of the LLM output (e.g., JSON). This ensures correct parsing. It is recommended not to change this format request.
- `json_output`: If `True`, the LLM must respond in JSON. Disabling this is not recommended. If you do, you will have to  
  change the `response_template` accordingly.


### Prompt Optimization (In developpement - better not to change anything)

This section enables **automatic prompt refinement** using a second LLM. It attempts to generate an improved version of the prompt to reduce classification errors.

- A second model (`llm2`) is used to review the prompt given to the first model (`llm1`) and suggest changes based on classification failures.
- If the new prompt performs better (fewer classification errors), it replaces the original.

**Warning**: This can lead to overfitting — the new prompt may work well on the training data but generalize poorly.  
It's highly recommended to **use a validation set** when using this feature.

### Prompt Optimization

- `provider_llm2`: LLM provider used for prompt improvement
- `model_name_llm2`: Name of the refinement model
- `temperature_llm2`: Temperature for the prompt-refiner LLM
- `max_iterations`: How many times the prompt should be revised.
For example, if you choose 3, each data entry will be classified three times: once with the original prompt, and twice with newly generated prompts.
- `use_validation_set`: Whether to use a separate validation set to monitor prompt overfitting (Boolean)
- `validation_size`: Number of samples in the validation set
- `random_state`: Random seed for reproducible train/validation split

### Majority vote

- `n_completions`: Number of completions per entry. 
It is possible to generate multiple responses for each entry using the same LLM. This will produce several classification labels for the same data point.
The final label is determined by majority vote. Generating multiple completions can improve robustness but also increases cost.

### Example

In the current example, we define two scenarios:

**Scenario 1**: Includes examples in the prompt (*few-shot*)

**Scenario 2**: Contains only the codebook and instructions (*zero-shot*)

In [5]:
scenarios = [
    {
        # LLM settings
        "provider_llm1": "openai",
        "model_name_llm1": "gpt-4o",
        "temperature_llm1": 0,
        "prompt_name": "few_shot",
        "subsample_size": 5,  # Size of data subset to use

        # Prompt configuration
        "template": """
You are an assistant that evaluates data entries.

The data has the following columns:
- "ID": Unique identifiant of the participant
- "Text": The reference text that participants must read beforehand. Their responses for the different steps must be semantically related to this text (same topic), but the answer to the question they are asking should not be found in the text.\n"
- "Identify": Response for the IDENTIFY step
- "Guess": Response for the GUESS step
- "Seek": Response for the SEEK step
- "Assess": Response for the ASSESS step

Here is an entry to evaluate:
{verbatim_text}

If a numeric value is present in the mechanical_rating column, copy it as the correct label.
If it’s empty, you’ll decide an overall cycle validity (0 or 1) based on the following codebook:

A cycle is considered valid if you can answer "yes" to all the following questions:

- Identify Step: Does the Identify step indicate a topic of interest?
- Guess Step: Does the Guess step suggest a possible explanation?
- Seek Step: Is the Seek step formulated as a question?
- Assess Step: Does it identify a possible answer or state that no answer where found ("no" is ok) ?
- Consistency: Are the Identify, Guess, and Seek steps related to the same question?
- Reference Link: Are the Identify, Guess, and Seek steps related to the topic of the reference text?
- Seek Question Originality: Is the answer to the Seek question not found (even vaguely) in the reference text?
- Resolving Answer: If the Assess step state an answer, does it answer to the question in the Seek step ?
- Valid Answer: If the ASSESS step indicates an answer was found, is the answer indeed in the assess_cues? → If not, then no answer was actually found, and the cycle is not valid.
- Valid No: If the ASSESS step indicates no answer was found, confirm that the answer to the SEEK question is not actually present in the assess_cues. → If the participant claims no answer was found, but it is in fact in assess_cues, the cycle is not valid.

Identify_validity, Guess_validity, Seek_validity, Assess_validity:
If one of those column already shows a numeric value (whatever the value), accept the step for this question without re-checking that step’s validity.

If all these criteria are met, the cycle is valid.
Validity is expressed as:
1: Valid cycle
0: Invalid cycle

Minor spelling, grammatical, or phrasing errors should not be penalized as long as the intent of the entry is clear and aligns with the inclusion criteria. Focus on the content and purpose of the entry rather than linguistic perfection.

Examples:

Example 1
Key:
AA25I4

Reference:
"Rain forms when water evaporates into the atmosphere, condenses into droplets, and falls due to gravity."

Cycle Steps:
IDENTIFY: "I don’t understand how rain forms."
GUESS: "Maybe rain condenses in the sky, forming droplets."
SEEK: "How does rain form?"
ASSESS: "No"
Assess Cues:

Validity Columns:
Identify_validity: NA
Guess_validity: 2
Seek_validity: NA
Assess_validity: NA
Mechanical_rating: NA

Reasoning
Since the mechanical_rating column is empty, the validity must be determined using the codebook.

Reasoning:
Identify step: Does the Identify step indicate a topic of interest?
Yes: The topic is the formation of rain.

Guess step: A numeric value is present in the Guess_validity column, so no further validation is needed.
Yes: It proposes condensation as the mechanism for rain formation.

Seek step: Is the Seek step formulated as a question?
Yes: It is explicitly phrased as a question with an interrogative structure.

Assess step: Does it identify a possible answer or state that no answer was found ("No" is acceptable)?
Yes: It states that the answer to the question was not found, which is a valid response in the Assess step.

Consistency: Are the Identify, Guess, and Seek steps related to the same topic?
Yes: They all pertain to the process of rain formation.

Reference Link: Are the Identify, Guess, and Seek steps related to the reference text?
Yes: The text discusses rain and explains its formation.

Seek Question Originality: Is the answer to the Seek question absent (even vaguely) from the reference text?
No: The answer is explicitly provided in the reference text.

Resolving Answer:
Not applicable (the answer was not found).

Valid Answer:
Not applicable (the answer was not found).

Valid No: Is the answer to the SEEK question absent from the assess_cues?
Yes: The answer to the SEEK question is not in assess_cues, so the "No" is valid.

Conclusion
The cycle is not valid because the answer to the SEEK question is explicitly present in the reference text.

Validity:
0
""",
        # Output
        "selected_fields": ["Classification", "Reasoning"],
        "prefix": "Classification",
        "label_type": "int",
        "response_template":
        """
Please follow the JSON format below:
```json
{{
  "Reasoning": "Your text here",
  "Classification": "Your integer here"
}}
""",
        "json_output": True,

        # Prompt optimization
        "provider_llm2": "openai",
        "model_name_llm2": "gpt-4o",
        "temperature_llm2": 0.7,
        "max_iterations": 1,
        "use_validation_set": False,
        "validation_size": 10,
        "random_state": 42,

        # Majority vote
        "n_completions": 1,

    },
    {
        # LLM settings
        "provider_llm1": "openai",
        "model_name_llm1": "gpt-4o",
        "temperature_llm1": 0,
        "prompt_name": "zero_shot",
        "subsample_size": 5,  # Size of data subset to use

        # Prompt configuration
        "template": """
You are an assistant that evaluates data entries.

The data has the following columns:
- "ID": Unique identifiant of the participant
- "Text": The reference text that participants must read beforehand. Their responses for the different steps must be semantically related to this text (same topic), but the answer to the question they are asking should not be found in the text.\n"
- "Identify": Response for the IDENTIFY step
- "Guess": Response for the GUESS step
- "Seek": Response for the SEEK step
- "Assess": Response for the ASSESS step

Here is an entry to evaluate:
{verbatim_text}

If a numeric value is present in the mechanical_rating column, copy it as the correct label.
If it’s empty, you’ll decide an overall cycle validity (0 or 1) based on the following codebook:

A cycle is considered valid if you can answer "yes" to all the following questions:

- Identify Step: Does the Identify step indicate a topic of interest?
- Guess Step: Does the Guess step suggest a possible explanation?
- Seek Step: Is the Seek step formulated as a question?
- Assess Step: Does it identify a possible answer or state that no answer where found ("no" is ok) ?
- Consistency: Are the Identify, Guess, and Seek steps related to the same question?
- Reference Link: Are the Identify, Guess, and Seek steps related to the topic of the reference text?
- Seek Question Originality: Is the answer to the Seek question not found (even vaguely) in the reference text?
- Resolving Answer: If the Assess step state an answer, does it answer to the question in the Seek step ?
- Valid Answer: If the ASSESS step indicates an answer was found, is the answer indeed in the assess_cues? → If not, then no answer was actually found, and the cycle is not valid.
- Valid No: If the ASSESS step indicates no answer was found, confirm that the answer to the SEEK question is not actually present in the assess_cues. → If the participant claims no answer was found, but it is in fact in assess_cues, the cycle is not valid.

Identify_validity, Guess_validity, Seek_validity, Assess_validity:
If one of those column already shows a numeric value (whatever the value), accept the step for this question without re-checking that step’s validity.

If all these criteria are met, the cycle is valid.
Validity is expressed as:
1: Valid cycle
0: Invalid cycle

Minor spelling, grammatical, or phrasing errors should not be penalized as long as the intent of the entry is clear and aligns with the inclusion criteria. Focus on the content and purpose of the entry rather than linguistic perfection.
""",
        # Output
        "selected_fields": ["Classification", "Reasoning"],
        "prefix": "Classification",
        "label_type": "int",
        "response_template":
        """
Please follow the JSON format below:
```json
{{
  "Reasoning": "Your text here",
  "Classification": "Your integer here"
}}
""",
        "json_output": True,

        # Prompt optimization
        "provider_llm2": "openai",
        "model_name_llm2": "gpt-4o",
        "temperature_llm2": 0.7,
        "max_iterations": 1,
        "use_validation_set": False,
        "validation_size": 10,
        "random_state": 42,

        # Majority vote
        "n_completions": 1,

    },
]

### Evaluation on Annotated Subset

Before launching the classification on the entire dataset, we first run it on the subset that has been manually annotated.  
This step allows us to compute performance metrics (e.g., **accuracy**, **F1-score**) by comparing LLM predictions to human labels,  
and therefore select which (if any) scenario can be used to classify the full, unlabeled dataset.

#### Configuration Parameters

- `annotation_columns`: The names of the columns containing human annotations.
- `labels`: The possible label values (in this case, `[0, 1]` for binary classification).

We filter out any rows with missing values in the annotation columns to ensure we're only evaluating on fully labeled data.

#### Repeated Runs for Stability

LLMs are **stochastic** by nature — even with a temperature of `0`, outputs can vary.  
To assess how consistent the model is, we introduce the `n_runs` parameter:

- `n_runs`: The number of times the classification is repeated for each scenario on the annotated data.

We recommend setting `n_runs = 3`, based on findings from **[Paper XX]** (insert reference),  
which showed that **three repetitions strike a good balance between stability and cost**.  
Running more times improves statistical reliability but increases costs proportionally.

#### `n_runs` vs `n_completions`

It’s important to distinguish between these two concepts:

- **`n_completions`**:  
  Controls how many responses are generated **within a single run** for each data point.  
  The final label is determined by **majority vote** over those completions.  
  **Example**:  
  If `n_completions = 3` and the model returns `[0, 0, 1]`, the selected label will be `0`.

- **`n_runs`**:  
  Repeats the **entire classification process** multiple times across the same data.  
  If you run the scenario three times and get `[0, 0, 1]` for a given entry,  
  that variation will be captured when calculating metrics (e.g., **variance**, **disagreement rate**).

In [6]:
# 9) Run scenarios and get results

annotation_columns = ['Rater_Oli', 'Rater_Gaia', 'Rater_Chloe']
labels = [0,1]

# Filter labeled data (drop rows with NaN in any annotation column)
labeled_data = data.dropna(subset=annotation_columns)
unlabeled_data = data[~data.index.isin(labeled_data.index)]

n_runs = 1  # Number of runs per scenario
verbose = True  # Whether to print verbose output

# Run the scenarios - this only runs the LLM and saves all the generated labels
complex_case_for_metrics = run_scenarios(
    scenarios=scenarios,
    data=labeled_data,
    annotation_columns=annotation_columns,
    labels=labels,
    n_runs=n_runs,
    verbose=verbose
)

Scenario 'few_shot' - Train size (all data): 5, No validation set

=== Processing Verbatim 1/5 ===
Prompt:

You are an assistant that evaluates data entries.

The data has the following columns:
- "ID": Unique identifiant of the participant
- "Text": The reference text that participants must read beforehand. Their responses for the different steps must be semantically related to this text (same topic), but the answer to the question they are asking should not be found in the text.
"
- "Identify": Response for the IDENTIFY step
- "Guess": Response for the GUESS step
- "Seek": Response for the SEEK step
- "Assess": Response for the ASSESS step

Here is an entry to evaluate:
Id: bc13_5

Text: Des scientifiques ont revele lexistence de traces dinsectes, de plantes et danimaux en Australie. Ces traces sont les restes danimaux et de vegetaux morts qui ont laisse leur empreinte dans la roche : c'est ce qu'on appelle des fossiles. Ils vivaient il y a plus de 11 millions dannees, bien avant lap

### Saving / Re-Loading the Results

This step provides an option to save the classification results to a file for future reference or further analysis.

In [ ]:
# Possibility to save the results

# Save the annotated results to a CSV file
complex_case_for_metrics.to_csv("data/multiclass_user_case/outputs/complex_case_for_metrics.csv", sep=";", index=False, encoding="utf-8-sig")

In [ ]:
# Optionally, load the annotated results from the CSV file if needed

complex_case_for_metrics = pd.read_csv(
    "data/outputs/complex_case_for_metrics.csv",
    sep=";",
    encoding="utf-8-sig"
)

## Evaluating Model Performance Against Human Annotations

To determine whether the model's classification is reliable and can be used to annotate the rest of the unlabeled dataset,  
it is recommended to evaluate its alignment with human annotations.  
If the alignment is sufficiently high, you may choose to rely on the model-generated labels for the remaining data.

We propose **four types of analysis**, depending on your goals:

- **If you want to measure agreement between annotators**:  
  Use **Cohen's Kappa**, a simple and widely used metric for inter-rater agreement.

- **If you need detailed per-class performance metrics** (e.g., recall, true positives, false positives):  
  Use **Classification Metrics**. This method gives a descriptive breakdown of model performance by class.

- **If you have multiple manual annotations and want a more robust estimate**:  
  Use **Krippendorff's Alpha**. This method provides:
  - A confidence interval for the agreement, computed via bootstrapping
  - An estimate of the risk that the true alpha value lies outside this interval

- **If you have multiple annotation columns (≥ 3)** and want to assess whether the model can "replace" or **outperform individual annotators**,  
  and you can afford to annotate 50–100 entries:  
  Use the **Alt-Test**. This stricter test compares the model to each annotator using a **leave-one-out** approach.

Among the available methods, **Krippendorff’s Alpha** and the **Alt-Test** are the ones we consider more **rigorous and robust**.

> **Note 1**: The final decision on whether the model's performance is “good enough” depends on your research domain,  
> acceptable error tolerance, and practical factors such as annotation cost and time. It can be totally valid to accept the model based solely on its Cohen’s kappa score,
 if it is approximately equivalent to human inter-rater agreement.

> **Note 2**: If the agreement between human annotators is low, the issue likely lies in the codebook (e.g., unclear guidelines) or the annotation task itself.
> In such cases, it’s unrealistic to expect the LLM to achieve high performance if humans themselves struggle to agree on the correct labels.

> **Note 3**: If you're not satisfied with the model’s performance, you can go back and **adjust the scenario** (this may include updating the codebook, adding examples, using another model...)  
> ⚠️ However, if you do this **multiple times**, it is strongly recommended to use a **validation set** to avoid overfitting to your annotated subset.

### Cohen's Kappa

This analysis provides:

- **Mean agreement between the LLM and all human annotators** (when multiple annotators are available)
- **Mean agreement among human annotators** (when multiple annotators are available)
- **Individual agreement scores** for all pairwise comparisons

#### Weighting Options

You can set kappa_weights to different values. Use:

- **unweighted (remove the parameter)**:  
  Treats all disagreements equally.  
  _Example: Disagreeing between `0` and `1` is treated the same as between `0` and `2`._

- **linear**:  
  Weights disagreements by their distance.  
  _Example: A disagreement between `0` and `2` is considered twice as bad as between `0` and `1`._

- **quadratic**:  
  Weights disagreements by the square of their distance.  
  _Example: A disagreement between `0` and `2` is considered four times as bad as between `0` and `1`._

> **Note **: If `n_runs` > 1, the reported metrics will include **variability across runs**, allowing you to assess the **consistency** of LLM performance.  
> Lower variance indicates more stable and reliable model behavior.

In [ ]:
# 10) Compute metrics from the detailed results
# First, compute kappa metrics

annotation_columns = ['Rater_Oli', 'Rater_Gaia', 'Rater_Chloe']
labels = [0,1]
verbose = True

kappa_df, detailed_kappa_metrics = compute_kappa_metrics(
    detailed_results_df=complex_case_for_metrics,
    annotation_columns=annotation_columns,
    labels=labels,
)

kappa_df


=== Columns in detailed_results_df (in compute_kappa_metrics) ===
['sample_id', 'split', 'verbatim', 'iteration', 'Rater_Oli', 'Rater_Gaia', 'Rater_Chloe', 'ModelPrediction', 'Reasoning', 'run', 'prompt_name', 'use_validation_set']


,prompt_name,iteration,n_runs,use_validation_set,N_train,N_val,accuracy_train,kappa_train,mean_llm_human_agreement,mean_human_human_agreement
0,few_shot,1,3,False,225,0,0.631111,0.271674,0.275054,0.876136
1,zero_shot,1,3,False,225,0,0.622222,0.257772,0.258639,0.876136


In [45]:
# Additional details about the kappa metrics

print("\n=== Detailed Kappa Metrics ===")
if detailed_kappa_metrics:
    for scenario_key, metrics in detailed_kappa_metrics.items():
        print(f"\nScenario: {scenario_key}")
        
        print("\nLLM vs Human Annotators:")
        print(metrics['llm_vs_human_df'])
        
        print("\nHuman vs Human Annotators:")
        print(metrics['human_vs_human_df'])
else:
    print("No detailed kappa metrics available.")


=== Detailed Kappa Metrics ===

Scenario: few_shot_iteration_1

LLM vs Human Annotators:
  Human_Annotator  Cohens_Kappa
0       Rater_Oli      0.251740
1      Rater_Gaia      0.251740
2     Rater_Chloe      0.321682

Human vs Human Annotators:
  Annotator_1  Annotator_2  Cohens_Kappa
0   Rater_Oli   Rater_Gaia      0.946429
1   Rater_Oli  Rater_Chloe      0.840989
2  Rater_Gaia  Rater_Chloe      0.840989

Scenario: zero_shot_iteration_1

LLM vs Human Annotators:
  Human_Annotator  Cohens_Kappa
0       Rater_Oli      0.240506
1      Rater_Gaia      0.240506
2     Rater_Chloe      0.294904

Human vs Human Annotators:
  Annotator_1  Annotator_2  Cohens_Kappa
0   Rater_Oli   Rater_Gaia      0.946429
1   Rater_Oli  Rater_Chloe      0.840989
2  Rater_Gaia  Rater_Chloe      0.840989


### Classification Metrics (Per-Class Analysis)

Analyze detailed classification metrics for each class, focusing on **recall** and **confusion matrix elements**.

This analysis uses the **majority vote from human annotations** as the ground truth and provides:

#### Global Metrics (prefix: `global_*`)

- `global_accuracy_train`: Overall accuracy on training data
- `global_recall_train`: Macro recall on training data
- `global_error_rate_train`: 1 - accuracy

(And similarly for validation data with suffix `_val`, if `use_validation_set = True`)

#### Per-Class Metrics (prefix: `class_<label>_*_train`)

For each class label (e.g., `0`, `1`), the following are computed:

- `class_<label>_recall_train`: Proportion of actual class instances correctly identified (True Positives)
- `class_<label>_error_rate_train`: Proportion of actual class instances incorrectly classified (Miss Rate)
- `class_<label>_correct_count_train`: Number of correctly predicted instances
- `class_<label>_missed_count_train`: Number of missed instances (False Negatives)
- `class_<label>_false_positives_train`: Number of incorrect predictions *as* this class (False Positives)


In [ ]:
# Compute classification metrics
classification_df = compute_classification_metrics_from_results(
    detailed_results_df=complex_case_for_metrics,
    annotation_columns=annotation_columns,
    labels=labels
)

pd.set_option("display.max_columns", None)    # show all columns
classification_df


=== Columns in detailed_results_df (in compute_classification_metrics_from_results) ===
['sample_id', 'split', 'verbatim', 'iteration', 'Rater_Oli', 'Rater_Gaia', 'Rater_Chloe', 'ModelPrediction', 'Reasoning', 'run', 'prompt_name', 'use_validation_set']


,prompt_name,iteration,n_runs,use_validation_set,N_train,N_val,global_accuracy_train,global_recall_train,global_error_rate_train,class_0_recall_train,class_0_error_rate_train,class_0_correct_count_train,class_0_missed_count_train,class_0_false_positives_train,class_1_recall_train,class_1_error_rate_train,class_1_correct_count_train,class_1_missed_count_train,class_1_false_positives_train
0,few_shot,1,3,False,225,0,0.631111,0.637821,0.368889,0.470085,0.529915,55,62,21,0.805556,0.194444,87,21,62
1,zero_shot,1,3,False,225,0,0.622222,0.631410,0.377778,0.401709,0.598291,47,70,15,0.861111,0.138889,93,15,70


### Krippendorff’s α Non‑Inferiority Test  
*(Requires ≥ 3 human annotation columns)*

#### Purpose

This test evaluates whether the model's annotations are **statistically non-inferior** to fully human-annotated data.  
If successful, this means the model can probably take over the annotation of the remaining, unlabeled data.

#### How the Test Works

- **Human reliability (`α_human`)**  
  Krippendorff’s α is computed across all *n* human annotators.

- **Model reliability (`α_model`)**  
  For each possible panel of (*n − 1*) humans + the model, compute Krippendorff’s α.  
  The final value is the **mean** α across all such combinations.

- **Effect size (Δ)**  
  \[
  \Delta = \alpha_{\text{model}} - \alpha_{\text{human}}
  \]  
  - Positive Δ → Model improves reliability  
  - Negative Δ → Performance drop

- **Uncertainty estimation via bootstrapping**  
  The dataset is resampled thousands of times (e.g., 2,000) to recompute Δ.  
  A **90 % confidence interval (CI)** (configurable) is constructed to show where the true Δ likely lies.


- **Non‑Inferiority Margin (`δ`)**
    You define `δ` (commonly set to **−0.05**) as the **largest acceptable drop** in α when using the model.

- **Decision rule**:  
  If the entire confidence interval lies **above `δ`**, the model is declared **non-inferior**.  
  With a 90 % CI, this reflects a **5 % one-sided risk** of wrongly approving a model worse than the lower born of the CI.

#### Interpretation Cheatsheet

| CI Position                 | What It Means for Deployment                                               |
|----------------------------|-----------------------------------------------------------------------------|
| CI fully above **0**       | ✅ Model is **statistically superior** to humans  |
| CI fully above **δ**, but crosses 0 | 🟡 Model is **non-inferior** (small, acceptable loss)     |
| CI touches or falls below **δ** | ❌ Model is possibly worse than the humans by the δ margin|

#### Why “5 % Risk”?

- A 90 % CI corresponds to a **one-sided α = 0.05** non-inferiority test.
- This 5 % risk applies to the **margin δ**, not to zero.
- If the CI just touches δ → ≈ 5 % chance that the **true Δ ≤ δ**
- If the CI is well above δ → Risk that **true Δ ≤ 0** is even lower than 5 %

#### Settings and Their Effects

| Setting                        | Increase →                          | Decrease →                          |
|-------------------------------|-------------------------------------|-------------------------------------|
| **Confidence level** (e.g. 90 % → 95 %) | – CI gets **wider**<br>– Test becomes **stricter**<br>– Type I error drops (5 % → 2.5 %) | – CI gets **narrower**<br>– Easier to declare non-inferiority<br>– Higher false positive risk |
| **Non-inferiority margin `δ`** (e.g. −0.05 → −0.10) | – You tolerate a **larger drop**<br>– Easier for model to pass<br>– Lower guaranteed quality | – You demand **closer match to humans**<br>– Harder to pass<br>– Stronger quality guarantee |


In [ ]:
# Run the non-inferiority test
non_inferiority_results = compute_krippendorff_non_inferiority(
    detailed_results_df=complex_case_for_metrics,
    annotation_columns=annotation_columns,
    model_column="ModelPrediction",
    level_of_measurement='ordinal',
    non_inferiority_margin=-0.05,
    n_bootstrap=2000, 
    confidence_level=90.0,
    random_seed=42, 
    verbose=False   
)

# Print results in a formatted way
print_non_inferiority_results(non_inferiority_results, show_per_run=False)


=== Non-inferiority Test: few_shot_iteration_1 ===
Human trios α: 0.8761 ± 0.0000
Model trios α: 0.4693 ± 0.0089
Δ = model − human = -0.4067 ± 0.0089
90% CI: [-0.5286, -0.2899]
Non-inferiority demonstrated in 0/3 runs
❌ Non-inferiority NOT demonstrated in any run (margin = -0.05)

=== Non-inferiority Test: zero_shot_iteration_1 ===
Human trios α: 0.8761 ± 0.0000
Model trios α: 0.4519 ± 0.0092
Δ = model − human = -0.4242 ± 0.0092
90% CI: [-0.5497, -0.3110]
Non-inferiority demonstrated in 0/3 runs
❌ Non-inferiority NOT demonstrated in any run (margin = -0.05)


### Alternative Annotator Test (ALT-Test)

The **ALT-Test** evaluates whether an LLM can perform **as well as or better than human annotators**, based on a **leave-one-human-out** approach.

This method requires **at least 3 human annotation columns**.

#### How It Works

- The LLM is compared against **each human annotator**, one at a time.
- For each comparison:
  - One human is **excluded**
  - The model’s predictions are evaluated **against the remaining human annotations**
  - This simulates a realistic setting where the LLM replaces a single annotator and is judged by agreement with the rest

#### Key Metrics in Output

- **`winning_rate_train`**: Proportion of annotators for which the LLM performs as well or better (after adjusting for ε)
- **`passed_alt_test_train`**: `True` if the LLM passes the test (i.e., `winning_rate ≥ 0.5`)
- **`avg_adv_prob_train`**: Average advantage probability, how likely the model is better across comparisons
- **`p_values_train`**: List of p-values for each comparison

#### Interpreting `ε` (Epsilon)

- `ε` accounts for the **cost/effort/time trade-off** between using an LLM and a human annotator.
- Higher `ε` gives the model more leeway, useful when **human annotations are costly**.
- Recommendations from the original paper:
  - `ε = 0.2` → when humans are **experts**
  - `ε = 0.1` → when humans are **crowdworkers**

> If `winning_rate ≥ 0.5`, the LLM is considered **statistically competitive with human annotators** for this dataset and scenario (the LLM is "better" than half the humans).

In [ ]:
# Run ALT test
epsilon = 0.2  # Epsilon parameter for ALT test
alt_test_df = run_alt_test_on_results(
    detailed_results_df=complex_case_for_metrics,
    annotation_columns=annotation_columns,
    labels=labels,
    epsilon=epsilon,
    alpha=0.05,
    verbose=verbose
)
alt_test_df = alt_test_df.drop(
    columns=["iteration", "run", "use_validation_set", "N_val", "n_runs"]
)

pd.set_option("display.max_colwidth", None)   # show full content in each cell
alt_test_df.tail(2)


=== Columns in detailed_results_df (in run_alt_test_on_results) ===
['sample_id', 'split', 'verbatim', 'iteration', 'Rater_Oli', 'Rater_Gaia', 'Rater_Chloe', 'ModelPrediction', 'Reasoning', 'run', 'prompt_name', 'use_validation_set']
=== ALT Test: Label Debugging ===
Label counts for each rater:
  ModelPrediction: 75 valid labels
  Rater_Oli: 75 valid labels
  Rater_Gaia: 75 valid labels
  Rater_Chloe: 75 valid labels

Label types for each rater:
  ModelPrediction: int64
  Rater_Oli: int64
  Rater_Gaia: int64
  Rater_Chloe: int64

Mixed types across raters: False

=== Converting labels to consistent types ===
Using label_type: int
Model predictions type after conversion: <class 'numpy.int32'>
Rater_Oli type after conversion: <class 'numpy.int32'>
Rater_Gaia type after conversion: <class 'numpy.int32'>
Rater_Chloe type after conversion: <class 'numpy.int32'>
=== Alt-Test: summary ===
P-values for each comparison:
Rater_Oli: p=0.9677 => rejectH0=False | rho_f=0.680, rho_h=0.987
Rater_Ga

,prompt_name,N_train,winning_rate_train,passed_alt_test_train,avg_adv_prob_train,p_values_train
6,few_shot,225,0.0,False,0.684444,"[0.9615964156138078, 0.9615964156138078, 0.9615964156138078]"
7,zero_shot,225,0.0,False,0.675556,"[0.971760465805921, 0.971760465805921, 0.971760465805921]"


## Final Step: Classify the Full Dataset

If you are satisfied with the evaluation metrics, you can now use the **best-performing scenario** to classify the **entire unlabeled dataset**.

Simply **copy the chosen scenario** and run the classification.

> This time, only **one run is needed**, since you're not computing evaluation metrics (there are no human labels to compare against).

If you're **not satisfied with the results**, feel free to continue exploring and testing **different scenarios**.

In [ ]:
scenario = [
    {
        # LLM settings
        "provider_llm1": "azure",
        "model_name_llm1": "gpt-4o",
        "temperature_llm1": 0,
        "prompt_name": "few_shot",
        "subsample_size": -1,  # Size of data subset to use

        # Prompt configuration
        "template": """
You are an assistant that evaluates data entries.

The data has the following columns:
- "ID": Unique identifiant of the participant
- "Text": The reference text that participants must read beforehand. Their responses for the different steps must be semantically related to this text (same topic), but the answer to the question they are asking should not be found in the text.\n"
- "Identify": Response for the IDENTIFY step
- "Guess": Response for the GUESS step
- "Seek": Response for the SEEK step
- "Assess": Response for the ASSESS step

Here is an entry to evaluate:
{verbatim_text}

If a numeric value is present in the mechanical_rating column, copy it as the correct label.
If it’s empty, you’ll decide an overall cycle validity (0 or 1) based on the following codebook:

A cycle is considered valid if you can answer "yes" to all the following questions:

- Identify Step: Does the Identify step indicate a topic of interest?
- Guess Step: Does the Guess step suggest a possible explanation?
- Seek Step: Is the Seek step formulated as a question?
- Assess Step: Does it identify a possible answer or state that no answer where found ("no" is ok) ?
- Consistency: Are the Identify, Guess, and Seek steps related to the same question?
- Reference Link: Are the Identify, Guess, and Seek steps related to the topic of the reference text?
- Seek Question Originality: Is the answer to the Seek question not found (even vaguely) in the reference text?
- Resolving Answer: If the Assess step state an answer, does it answer to the question in the Seek step ?
- Valid Answer: If the ASSESS step indicates an answer was found, is the answer indeed in the assess_cues? → If not, then no answer was actually found, and the cycle is not valid.
- Valid No: If the ASSESS step indicates no answer was found, confirm that the answer to the SEEK question is not actually present in the assess_cues. → If the participant claims no answer was found, but it is in fact in assess_cues, the cycle is not valid.

Identify_validity, Guess_validity, Seek_validity, Assess_validity:
If one of those column already shows a numeric value (whatever the value), accept the step for this question without re-checking that step’s validity.

If all these criteria are met, the cycle is valid.
Validity is expressed as:
1: Valid cycle
0: Invalid cycle

Minor spelling, grammatical, or phrasing errors should not be penalized as long as the intent of the entry is clear and aligns with the inclusion criteria. Focus on the content and purpose of the entry rather than linguistic perfection.

Examples:

Example 1
Key:
AA25I4

Reference:
"Rain forms when water evaporates into the atmosphere, condenses into droplets, and falls due to gravity."

Cycle Steps:
IDENTIFY: "I don’t understand how rain forms."
GUESS: "Maybe rain condenses in the sky, forming droplets."
SEEK: "How does rain form?"
ASSESS: "No"
Assess Cues:

Validity Columns:
Identify_validity: NA
Guess_validity: 2
Seek_validity: NA
Assess_validity: NA
Mechanical_rating: NA

Reasoning
Since the mechanical_rating column is empty, the validity must be determined using the codebook.

Reasoning:
Identify step: Does the Identify step indicate a topic of interest?
Yes: The topic is the formation of rain.

Guess step: A numeric value is present in the Guess_validity column, so no further validation is needed.
Yes: It proposes condensation as the mechanism for rain formation.

Seek step: Is the Seek step formulated as a question?
Yes: It is explicitly phrased as a question with an interrogative structure.

Assess step: Does it identify a possible answer or state that no answer was found ("No" is acceptable)?
Yes: It states that the answer to the question was not found, which is a valid response in the Assess step.

Consistency: Are the Identify, Guess, and Seek steps related to the same topic?
Yes: They all pertain to the process of rain formation.

Reference Link: Are the Identify, Guess, and Seek steps related to the reference text?
Yes: The text discusses rain and explains its formation.

Seek Question Originality: Is the answer to the Seek question absent (even vaguely) from the reference text?
No: The answer is explicitly provided in the reference text.

Resolving Answer:
Not applicable (the answer was not found).

Valid Answer:
Not applicable (the answer was not found).

Valid No: Is the answer to the SEEK question absent from the assess_cues?
Yes: The answer to the SEEK question is not in assess_cues, so the "No" is valid.

Conclusion
The cycle is not valid because the answer to the SEEK question is explicitly present in the reference text.

Validity:
0
""",
        # Output
        "selected_fields": ["Classification", "Reasoning"],
        "prefix": "Classification",
        "label_type": "int",
        "response_template":
        """
Please follow the JSON format below:
```json
{{
  "Reasoning": "Your text here",
  "Classification": "Your integer here"
}}
""",
        "json_output": True,

        # Prompt optimization
        "provider_llm2": "azure",
        "model_name_llm2": "gpt-4o",
        "temperature_llm2": 0.7,
        "max_iterations": 1,
        "use_validation_set": False,
        "validation_size": 10,
        "random_state": 42,

        # Majority vote
        "n_completions": 1,

    },
]

# Run the scenario
complex_case_fully_annotated = run_scenarios(
    scenarios=scenario,
    data=data,
    annotation_columns=annotation_columns,
    labels=labels,
    n_runs=n_runs,
    verbose=verbose
)

In [ ]:
complex_case_fully_annotated.to_csv("data/outputs/complex_case_fully_annotated.csv", sep=";", index=False, encoding="utf-8-sig")